In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm

In [2]:
store = pd.read_csv("data/commercial_with_xy.csv",encoding = "UTF-8")
store.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,집객시설_수,아파트_평균_면적,아파트_평균_시가,상권_변화_지표,...,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,구분,점포_수,매출_금액,매출_건수,엑스좌표_값,와이좌표_값
0,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,개인_서비스업,116,3.230693e+07,398,196193,456641
1,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,교육_및_사업자원_서비스업,128,2.660860e+07,166,196193,456641
2,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,기타_도소매업,98,2.564806e+06,96,196193,456641
3,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,생활용품_도소매업,151,3.070350e+07,358,196193,456641
4,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,수리업,24,1.999051e+08,874,196193,456641


In [3]:
store.columns

Index(['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '총_생활인구_수',
       '집객시설_수', '아파트_평균_면적', '아파트_평균_시가', '상권_변화_지표', '상권_변화_지표_명',
       '운영_영업_개월_평균', '폐업_영업_개월_평균', '서울_운영_영업_개월_평균', '서울_폐업_영업_개월_평균', '구분',
       '점포_수', '매출_금액', '매출_건수', '엑스좌표_값', '와이좌표_값'],
      dtype='object')

In [4]:
store = store[['상권_코드_명','총_생활인구_수','집객시설_수','점포_수','아파트_평균_면적','아파트_평균_시가','서울_운영_영업_개월_평균','서울_폐업_영업_개월_평균','매출_건수', "매출_금액",'구분']]

In [5]:
store = store.drop_duplicates()

연도별값이 중요하지 않아서 연도 column 제외 + 그로인해 생기는 duplicate되는 값 제거

In [6]:
store

,상권_코드_명,총_생활인구_수,집객시설_수,점포_수,아파트_평균_면적,아파트_평균_시가,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,매출_건수,매출_금액,구분
0,이북5도청사,139405.75,22.5,116,124.0,376996219.5,98.75,51.25,398,3.230693e+07,개인_서비스업
1,이북5도청사,139405.75,22.5,128,124.0,376996219.5,98.75,51.25,166,2.660860e+07,교육_및_사업자원_서비스업
2,이북5도청사,139405.75,22.5,98,124.0,376996219.5,98.75,51.25,96,2.564806e+06,기타_도소매업
3,이북5도청사,139405.75,22.5,151,124.0,376996219.5,98.75,51.25,358,3.070350e+07,생활용품_도소매업
4,이북5도청사,139405.75,22.5,24,124.0,376996219.5,98.75,51.25,874,1.999051e+08,수리업
...,...,...,...,...,...,...,...,...,...,...,...
105310,중부시장(신중부시장),927336.00,18.0,17,15.0,148636943.0,121.25,55.50,308,3.510748e+07,숙박업
105311,중부시장(신중부시장),927336.00,18.0,25,15.0,148636943.0,121.25,55.50,375,1.061767e+07,스포츠_및_오락_서비스업
105312,중부시장(신중부시장),927336.00,18.0,1663,15.0,148636943.0,121.25,55.50,147094,7.184864e+09,식료품_도소매업
105313,중부시장(신중부시장),927336.00,18.0,243,15.0,148636943.0,121.25,55.50,19070,5.655092e+08,음식점_및_주점업


In [7]:
store.rename(columns={'서울_운영_영업_개월_평균': '영업_개월_평균', '서울_폐업_영업_개월_평균':'폐업_개월_평균','구분':'업종_명'},inplace=True)

In [8]:
store = store.reset_index(drop=True)

In [9]:
store

,상권_코드_명,총_생활인구_수,집객시설_수,점포_수,아파트_평균_면적,아파트_평균_시가,영업_개월_평균,폐업_개월_평균,매출_건수,매출_금액,업종_명
0,이북5도청사,139405.75,22.5,116,124.0,376996219.5,98.75,51.25,398,3.230693e+07,개인_서비스업
1,이북5도청사,139405.75,22.5,128,124.0,376996219.5,98.75,51.25,166,2.660860e+07,교육_및_사업자원_서비스업
2,이북5도청사,139405.75,22.5,98,124.0,376996219.5,98.75,51.25,96,2.564806e+06,기타_도소매업
3,이북5도청사,139405.75,22.5,151,124.0,376996219.5,98.75,51.25,358,3.070350e+07,생활용품_도소매업
4,이북5도청사,139405.75,22.5,24,124.0,376996219.5,98.75,51.25,874,1.999051e+08,수리업
...,...,...,...,...,...,...,...,...,...,...,...
105193,중부시장(신중부시장),927336.00,18.0,17,15.0,148636943.0,121.25,55.50,308,3.510748e+07,숙박업
105194,중부시장(신중부시장),927336.00,18.0,25,15.0,148636943.0,121.25,55.50,375,1.061767e+07,스포츠_및_오락_서비스업
105195,중부시장(신중부시장),927336.00,18.0,1663,15.0,148636943.0,121.25,55.50,147094,7.184864e+09,식료품_도소매업
105196,중부시장(신중부시장),927336.00,18.0,243,15.0,148636943.0,121.25,55.50,19070,5.655092e+08,음식점_및_주점업


In [10]:
store["업종_명"].unique()

array(['개인_서비스업', '교육_및_사업자원_서비스업', '기타_도소매업', '생활용품_도소매업', '수리업', '숙박업',
       '스포츠_및_오락_서비스업', '식료품_도소매업', '음식점_및_주점업', '제조업',
       '가전제품_및_통신기기_도소매업', '기타_서비스업', '부동산_및_임대업', '중고상품_도소매업'],
      dtype=object)

In [11]:
sectors = ["개인_서비스업", "교육_및_사업자원_서비스업", "기타_도소매업", "생활용품_도소매업", "수리업", "숙박업",
          "스포츠_및_오락_서비스업", "식료품_도소매업", "음식점_및_주점업", "제조업", "가전제품_및_통신기기_도소매업",
          "기타_서비스업", "부동산_및_임대업", "중고상품_도소매업"]

In [12]:
ols_summary = []
for sector in sectors:
    data = store[store["업종_명"] == sector]
    X = data[["총_생활인구_수", "집객시설_수", "점포_수", "아파트_평균_면적", "아파트_평균_시가", "영업_개월_평균", "폐업_개월_평균", "매출_건수"]]
    y = data["매출_금액"]
    result = sm.OLS(y, X).fit()

    ols_summary.append(result.summary())

In [13]:
ols_summary[10]

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  매출_금액   R-squared (uncentered):                   0.630
Model:                            OLS   Adj. R-squared (uncentered):              0.630
Method:                 Least Squares   F-statistic:                              1737.
Date:                Tue, 23 May 2023   Prob (F-statistic):                        0.00
Time:                        03:23:38   Log-Likelihood:                     -1.9452e+05
No. Observations:                8166   AIC:                                  3.891e+05
Df Residuals:                    8158   BIC:                                  3.891e+05
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
총_생활인구_수     -41.1826     45.155     -0.912      0.362    -129.698      47.333
집객시설_수     -1.359e+07   1.99e+06     -6.839      0.000   -1.75e+07   -9.69e+06
점포_수        1.505e+07   1.47e+05    102.173      0.000    1.48e+07    1.53e+07
아파트_평균_면적   1.118e+06   5.64e+06      0.198      0.843   -9.94e+06    1.22e+07
아파트_평균_시가     -0.0714      0.607     -0.118      0.906      -1.262       1.119
영업_개월_평균    9.361e+07   1.48e+07      6.315      0.000    6.45e+07    1.23e+08
폐업_개월_평균     -2.1e+08   3.21e+07     -6.551      0.000   -2.73e+08   -1.47e+08
매출_건수       1.669e+04    789.879     21.131      0.000    1.51e+04    1.82e+04
==============================================================================
Omnibus:                    16292.107   Durbin-Watson:                   0.819
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        215676473.458
Skew:                          15.435   Prob(JB):                         0.00
Kurtosis:                     798.565   Cond. No.                     1.60e+08
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.6e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

In [15]:
data = store[store["업종_명"] == "가전제품_및_통신기기_도소매업"]

X = data[["총_생활인구_수", "집객시설_수", "점포_수", "아파트_평균_면적", "아파트_평균_시가", "영업_개월_평균", "폐업_개월_평균", "매출_건수"]]
y = data["매출_금액"]

X = scale(X)
y = scale(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

svr = SVR(kernel='rbf', gamma='auto')
y_rbf = svr.fit(X_train,y_train)

score = svr.score(X_train, y_train)
print("R-squared: ", score)

cv_score = cross_val_score(svr, X, y, cv=5)
print("CV mean score: ", cv_score.mean())

y_pred = svr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


R-squared:  0.17751228839415023
CV mean score:  -0.09184938422876425
MSE:  1.0582018500389387


In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
data = store[store["업종_명"] == "가전제품_및_통신기기_도소매업"]

X = data[["총_생활인구_수", "집객시설_수", "점포_수", "아파트_평균_면적", "아파트_평균_시가", "영업_개월_평균", "폐업_개월_평균", "매출_건수"]]
y = data["매출_금액"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = RandomForestRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_absolute_error(y_test, y_pred)
print('mse: ', mse)
rmse = (np.sqrt(mse))
print('rmse: ', rmse)
r2 = r2_score(y_test, y_pred)
print('R-squared: ', r2)

mse:  581534407.1060553
rmse:  24115.024509754396
R-squared:  0.735112943458283


In [18]:
import xgboost

data = store[store["업종_명"] == "가전제품_및_통신기기_도소매업"]

X = data[["총_생활인구_수", "집객시설_수", "점포_수", "아파트_평균_면적", "아파트_평균_시가", "영업_개월_평균", "폐업_개월_평균", "매출_건수"]]
y = data["매출_금액"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

xgb_model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0,
                                 subsample=0.75, colsample_bytree=1, max_depth=7)

xgb_model.fit(X_train,y_train)

r2 = xgb_model.score(X_train, y_train)
print(r2)

ModuleNotFoundError: No module named 'xgboost'